<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Import-Packages" data-toc-modified-id="Import-Packages-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Import Packages</a></span></li><li><span><a href="#Loading-and-Cleaning-Data" data-toc-modified-id="Loading-and-Cleaning-Data-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Loading and Cleaning Data</a></span><ul class="toc-item"><li><span><a href="#Load-Data" data-toc-modified-id="Load-Data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Load Data</a></span></li><li><span><a href="#Clean-Raw-Text" data-toc-modified-id="Clean-Raw-Text-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Clean Raw Text</a></span></li><li><span><a href="#Character-Mappings" data-toc-modified-id="Character-Mappings-2.3"><span class="toc-item-num">2.3&nbsp;&nbsp;</span>Character Mappings</a></span></li></ul></li><li><span><a href="#Creating-train-and-test-data-objects" data-toc-modified-id="Creating-train-and-test-data-objects-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Creating train and test data objects</a></span><ul class="toc-item"><li><span><a href="#Dataset-Generation-Functions" data-toc-modified-id="Dataset-Generation-Functions-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Dataset Generation Functions</a></span></li><li><span><a href="#Creating-Datasets" data-toc-modified-id="Creating-Datasets-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Creating Datasets</a></span></li></ul></li><li><span><a href="#Models" data-toc-modified-id="Models-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Models</a></span><ul class="toc-item"><li><span><a href="#750-Sequence-Length-Model" data-toc-modified-id="750-Sequence-Length-Model-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>750 Sequence Length Model</a></span><ul class="toc-item"><li><span><a href="#Parameters" data-toc-modified-id="Parameters-4.1.1"><span class="toc-item-num">4.1.1&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Set-up-Checkpoint-Path" data-toc-modified-id="Set-up-Checkpoint-Path-4.1.2"><span class="toc-item-num">4.1.2&nbsp;&nbsp;</span>Set up Checkpoint Path</a></span></li><li><span><a href="#Build-model" data-toc-modified-id="Build-model-4.1.3"><span class="toc-item-num">4.1.3&nbsp;&nbsp;</span>Build model</a></span></li><li><span><a href="#Fit-Model" data-toc-modified-id="Fit-Model-4.1.4"><span class="toc-item-num">4.1.4&nbsp;&nbsp;</span>Fit Model</a></span></li><li><span><a href="#1-Batch-Model-and-Imputing-Redactions" data-toc-modified-id="1-Batch-Model-and-Imputing-Redactions-4.1.5"><span class="toc-item-num">4.1.5&nbsp;&nbsp;</span>1 Batch Model and Imputing Redactions</a></span></li></ul></li><li><span><a href="#400-Sequence-Length-Model" data-toc-modified-id="400-Sequence-Length-Model-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>400 Sequence Length Model</a></span><ul class="toc-item"><li><span><a href="#Parameters" data-toc-modified-id="Parameters-4.2.1"><span class="toc-item-num">4.2.1&nbsp;&nbsp;</span>Parameters</a></span></li><li><span><a href="#Set-up-Checkpoint-Path" data-toc-modified-id="Set-up-Checkpoint-Path-4.2.2"><span class="toc-item-num">4.2.2&nbsp;&nbsp;</span>Set up Checkpoint Path</a></span></li><li><span><a href="#Build-Model" data-toc-modified-id="Build-Model-4.2.3"><span class="toc-item-num">4.2.3&nbsp;&nbsp;</span>Build Model</a></span></li><li><span><a href="#Fit-Model" data-toc-modified-id="Fit-Model-4.2.4"><span class="toc-item-num">4.2.4&nbsp;&nbsp;</span>Fit Model</a></span></li><li><span><a href="#1-Batch-and-Redactions" data-toc-modified-id="1-Batch-and-Redactions-4.2.5"><span class="toc-item-num">4.2.5&nbsp;&nbsp;</span>1 Batch and Redactions</a></span></li></ul></li></ul></li></ul></div>

## Import Packages

The model will be made in TensorFlow using [Eager Execution mode](https://www.tensorflow.org/guide/eager). 

In [1]:
#from __future__ import absolute_import, division, print_function, unicode_literals
import re
import numpy as np
import os
import time
import tensorflow as tf
from tensorflow.python.keras.optimizer_v2.adam import Adam
tf.compat.v1.enable_eager_execution()

## Loading and Cleaning Data


### Load Data
I frequently trained models using google colab so I have an option to mount and load a google drive folder. 

In [2]:
### Option to run notebook using google colab as data storage
drive_load = False

In [3]:
if drive_load == True:
    from google.colab import drive
    drive.mount('/content/gdrive')
    
    path_to_file = "/content/gdrive/My Drive/code/redaction model/strippedtext.txt"
else:
    path_to_file = "strippedtext.txt"


raw_text = open(path_to_file, 'rb').read().decode(encoding='utf-8')
unfiltered_vocab = sorted(set(raw_text))
print(unfiltered_vocab)

['\n', '\r', ' ', '!', '"', '#', '$', '%', '&', "'", '(', ')', '*', '+', ',', '-', '.', '/', '0', '1', '2', '3', '4', '5', '6', '7', '8', '9', ':', ';', '<', '=', '>', '?', '@', 'A', 'B', 'C', 'D', 'E', 'F', 'G', 'H', 'I', 'J', 'K', 'L', 'M', 'N', 'O', 'P', 'Q', 'R', 'S', 'T', 'U', 'V', 'W', 'X', 'Y', 'Z', '[', '\\', ']', '_', 'a', 'b', 'c', 'd', 'e', 'f', 'g', 'h', 'i', 'j', 'k', 'l', 'm', 'n', 'o', 'p', 'q', 'r', 's', 't', 'u', 'v', 'w', 'x', 'y', 'z', '|', '§', '¶', 'é', 'ü', 'ſ', 'а', 'в', 'д', 'е', 'и', 'к', 'м', 'н', 'о', 'п', 'р', 'т', 'ч', 'ы', 'я', '–', '—', '‘', '’', '“', '”', '…', '■']


### Clean Raw Text

Remove a bunch of specific characters from the text. 

In [4]:
print('Length of text pre-substituion : {} characters.'.format(len(raw_text)))
print('{} unique characters after substitution'.format(len(unfiltered_vocab)))

sub_text = re.sub('[!@#$§\[\]%&+*\(\);<>=|¶éüſавдеикмнопртчыя–—‘’“”]', '', raw_text)
sub_text = re.sub('…', '...', sub_text)

vocab = sorted(set(sub_text))
vocab_size = len(vocab)

print('\nLength of text post-substituion : {} characters. '.format(len(sub_text)))
print('{} unique characters after substitution. \n\nPre-to-Post difference : {} characters.'.format(len(vocab), len(raw_text) - len(sub_text)))

Length of text pre-substituion : 1320770 characters.
120 unique characters after substitution

Length of text post-substituion : 1304546 characters. 
76 unique characters after substitution. 

Pre-to-Post difference : 16224 characters.


### Character Mappings

Create mappings from characters to indices and from indices to characters. 

In [5]:
### Create a mapping from unique characters to indices and vice versa
char2idx = {u:i for i, u in enumerate(vocab)}
idx2char = np.array(vocab)

for char,_ in zip(char2idx, range(5)):
    print('  {:4s}: {:3d},'.format(repr(char), char2idx[char]))

  '\n':   0,
  '\r':   1,
  ' ' :   2,
  '"' :   3,
  "'" :   4,


## Creating train and test data objects

Create the training and test text sets.
- The training set will be all the text from the original report with any redactions sections removed and replaced with three periods (...).
- The test set will be made of all the redactions in the report and some number of preceding characters to prompt the model. 

In [8]:
train_text = re.sub('■+', '...',sub_text)

### Each redaction has the first and last index values recorded along with the 
### length of the redactions
### Length of redactions isn't actually used anywhere
redactions = [[m.start(), m.end(), m.end() - m.start()] for m in re.finditer('■+', sub_text)]

prec_char_750 = 750
prec_char_400 = 400

test_text_750 = [sub_text[i[0] - prec_char_750 : i[1]] for i in redactions]
test_text_400 = [sub_text[i[0] - prec_char_400 : i[1]] for i in redactions]

print(test_text_400[:2])

['the earliest Russian interference operations identified by the investigation  a social media campaign designed to provoke and amplify political and social discord in the United States. The IRA was based in St. Petersburg, Russia, and received funding from Russian oligarch Yevgeniy Prigozhin and companies he controlled. Prigozhin is widely reported to have ties to Russian President Vladimir Putin, ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■', 'ng from Russian oligarch Yevgeniy Prigozhin and companies he controlled. Prigozhin is widely reported to have ties to Russian President Vladimir Putin, ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■\r\nIn mid-2014, the IRA sent employees to the United States on an intelligence-gathering mission with instructions ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■

Convert the text into the numeric array form using the character to index mappings made earlier. 

In [9]:
text_as_int = np.array([char2idx[c] for c in sub_text])
train_text_as_int = np.array([char2idx[c] for c in train_text])
test_text_750_as_int = [np.array([char2idx[c] for c in i]) for i in test_text_750]
test_text_400_as_int = [np.array([char2idx[c] for c in i]) for i in test_text_400]

### Dataset Generation Functions

Create a function to chunk up an input string, splitting it into predictor text and a target character. 

Batch the full text into seq_length + 1 length sequences then `map` that new function onto the sequences to produce predictors and target chunks.

In [10]:
def split_input_target(chunk):
    input_text = chunk[:-1]
    target_text = chunk[1:]
    return input_text, target_text

def dataset_generator(seq_length, text):
    char_dataset = tf.data.Dataset.from_tensor_slices(train_text_as_int)
    
    ### Increasing to 100 prevented model from "running out of data". 
    ### Probably part of ResourceExhausted Error but running out of data was worse.
    repeat_n = 100
    sequences = char_dataset.repeat(repeat_n).batch(seq_length+1, drop_remainder=True)
    dataset = sequences.map(split_input_target)
    return dataset

Create a dataset with sequence lengths 750 and 400 to test out two different models.

### Creating Datasets

In [11]:
seq_length_400 = 400
dataset_400 = dataset_generator(seq_length_400, sub_text)

seq_length_750 = 750
dataset_750 = dataset_generator(seq_length_750, sub_text)
print(dataset_400, '\n', dataset_750)

<DatasetV1Adapter shapes: ((400,), (400,)), types: (tf.int64, tf.int64)> 
 <DatasetV1Adapter shapes: ((750,), (750,)), types: (tf.int64, tf.int64)>


## Models

First I'll define some variables and functions that'll be used in both models.

In [12]:
BUFFER_SIZE = 10000
BATCH_SIZE = 64

Define the loss function for the models. We'll use categorical crossentropy as that's the simplest loss function for our problem. 

In [13]:
def loss(labels, logits):
    return tf.keras.losses.sparse_categorical_crossentropy(labels, logits, from_logits=True)

Function to generate text given a starting string using a model. Code originally taken from [this](https://github.com/tensorflow/docs/blob/master/site/en/r2/tutorials/text/text_generation.ipynb) TensorFlow tutorial but adapted to this use case.

- Changes include:
    - Making `temperature` a parameter
    - Changing `num_generate` to dynamically set to the length of redacted piece of `start_string`
    - Add a separator in the text generated to show where redaction/prediction starts.

In [14]:
def generate_text(model, start_string, temperature=1):
    # Evaluation step (generating text using the learned model)

    # Number of characters to generate
    num_generate = len(start_string) - len(re.sub('■+', '', start_string))
    start_string = re.sub('■+', '', start_string)
    # Converting our start string to numbers (vectorizing)
    input_eval = [char2idx[s] for s in start_string]
    input_eval = tf.expand_dims(input_eval, 0)

    # Empty string to store our results
    text_generated = []

    # Low temperatures results in more predictable text.
    # Higher temperatures results in more surprising text.
    # Experiment to find the best setting.
    #temperature = 1.0

    # Here batch size == 1
    model.reset_states()
    for i in range(num_generate):
        predictions = model(input_eval)
        # remove the batch dimension
        predictions = tf.squeeze(predictions, 0)

        # using a categorical distribution to predict the word returned by the model
        predictions = predictions / temperature
        predicted_id = tf.random.categorical(predictions, num_samples=1)[-1,0].numpy()

        # We pass the predicted word as the next input to the model
        # along with the previous hidden state
        input_eval = tf.expand_dims([predicted_id], 0)

        text_generated.append(idx2char[predicted_id])

    return (start_string + ' || ' + ''.join(text_generated))

### 750 Sequence Length Model

#### Parameters

Set parameters for 750 sequence length model. 

In [16]:
learning_rate_750 = 5e-4
embedding_dim_750 = 256
nb_epoches_750 = 1
rnn_units_750 = 600
load_weights_750 = True
keep_training_750 = False

### Not sure if is doing what I think but it was used in a tutorial I followed so I'm using it
### Essentially, I want each epoch to read the document once
examples_per_epoch_750 = len(sub_text)//seq_length_750
steps_per_epoch_750 = int(examples_per_epoch_750 / BATCH_SIZE)

### Set up dataset that's been shuffled and batched
dataset_sb_750 = dataset_750.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset_sb_750)

<DatasetV1Adapter shapes: ((64, 750), (64, 750)), types: (tf.int64, tf.int64)>


Whether the user is loading from drive or now, pick the folder that'll have the checkpoints in it.

#### Set up Checkpoint Path

In [17]:
if drive_load == True:
    checkpoint_750_path = "/content/gdrive/My Drive/code/redaction model/checkpoints_750_model/epochs:{epoch:03d}-loss:{loss:.3f}.hdf5"
else:
    checkpoint_750_path = "checkpoints_750_model/epochs:{epoch:03d}-loss:{loss:.3f}.hdf5"
    
checkpoint_750_dir = os.path.dirname(checkpoint_750_path)

### Make the directory if it doesn't exist
if not os.path.isdir(checkpoint_750_dir):
    os.mkdir(checkpoint_750_dir)
    
print(checkpoint_750_dir)

checkpoints_750_model


#### Build model
Build the model for the 750 sequence length model. 

Use only 1 GRU because that's as big as Google Colab will let it get while in Eager Execution mode. 

In [18]:
def build_model_750(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.5), 
        tf.keras.layers.Dense(2*vocab_size),
        tf.keras.layers.Dense(vocab_size)])
    return model

Make the model object, compile it and potentially load weights to it if necessary.

In [19]:
model_750 = build_model_750(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim_750,
    rnn_units=rnn_units_750,
    batch_size=BATCH_SIZE)

adam_opt = Adam(lr=learning_rate_750)

model_750.compile(optimizer=adam_opt, loss=loss)

most_recent_750_checkpoint = max([checkpoint_750_dir + '/' + i for i in os.listdir(checkpoint_750_dir)], key = os.path.getctime)

if load_weights_750 == True:
    model_750.load_weights(most_recent_750_checkpoint)
    print("Weights loaded from : ", most_recent_750_checkpoint)

model_750.summary()


Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Weights loaded from :  checkpoints_750_model/epochs:001-loss:1.338.hdf5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (64, None, 256)           19456     
_________________________________________________________________
gru (GRU)                    (64, None, 600)           1542600   
_________________________________________________________________
dropout (Dropout)            (64, None, 600)           0         
_________________________________________________________________
dense (Dense)                (64, None, 152)           91352     
_________________________________________________________________
dense_1 (Dense)              (64, None, 76)            11628     
Total params: 1

#### Fit Model

Add callbacks then fit the model.
- If `load_weights` == True then the user has weights they would like to load prior to training.
- If `keep_training` == True then the user wants the model to train further after starting from a checkpoint.

In [25]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_750_path, period=1,
                                                         monitor='loss', save_best_only=True, 
                                                         mode='min')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

callbacks_list = [checkpoint_callback, earlystop_callback]

if load_weights_750 == True:
    if keep_training_750 == True:
        history = model_750.fit(
                            dataset_sb_750, 
                            epochs=nb_epoches_750, 
                            steps_per_epoch=steps_per_epoch_750, 
                            callbacks=callbacks_list)
    elif keep_training_750 == False:
        history = model_750.fit(
                            dataset_sb_750, 
                            epochs=1, 
                            steps_per_epoch=1, 
                            callbacks=callbacks_list)
else:
    history = model_750.fit(
                        dataset_sb_750, 
                        epochs=nb_epoches_750, 
                        steps_per_epoch=steps_per_epoch_750, 
                        callbacks=callbacks_list)

1/1 [==============================] - 18s 18s/step - loss: 1.3089


#### 1 Batch Model and Imputing Redactions

Build a model with 1 batch size and load weights from above cell's checkpoints. 

Using this 1batch model, generate predictions for the first 5 redactions of the Mueller report.
- Generate three predictions with varying temperatures per redaction.
    - Lower temperature gives less predictable predictions.

In [45]:
print(most_recent_750_checkpoint)

model_750_1batch = build_model_750(vocab_size, embedding_dim_750, rnn_units_750, batch_size=1)
model_750_1batch.load_weights(most_recent_750_checkpoint)
model_750_1batch.build(tf.TensorShape([1, None]))
model_750_1batch.summary()

print("750 Sequence Length predictors")
for index, redaction in enumerate(test_text_750[:10]):
    print("\n################################################\n")
    print(index)
    print("\n### : Original Text")
    print(redaction)
    print("\n### : Temperature == 1 prediction")
    print(generate_text(model_750_1batch, start_string=redaction, temperature=1))
    print("\n### : Temperature == .6 prediction")
    print(generate_text(model_750_1batch, start_string=redaction, temperature=.6))
    print("\n### : Temperature == 1.3 prediction")
    print(generate_text(model_750_1batch, start_string=redaction, temperature=1.3))
        

checkpoints_750_model/epochs:001-loss:1.338.hdf5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_9 (Embedding)      (1, None, 256)            19456     
_________________________________________________________________
gru_13 (GRU)                 (1, None, 600)            1542600   
_________________________________________________________________
dropout_13 (Dropout)         (1, None, 600)            0         
_________________________________________________________________
dense_18 (Dense)             (1, None, 152)            91352     
_________________________________________________________________
dense_19 (Dense)             (1, None, 76)             11628     
Total params: 1,665,036
Trainable params: 1,665,036
Non-trainable params: 0
_________________________________________________________________
750 Sequence Length predictors

################################################

0

TIVE SUMMARY TO VOLUME I
RUSSIAN SOCIAL MEDIA CAMPAIGN
The Internet Research Agency IRA carried out the earliest Russian interference operations identified by the investigation  a social media campaign designed to provoke and amplify political and social discord in the United States. The IRA was based in St. Petersburg, Russia, and received funding from Russian oligarch Yevgeniy Prigozhin and companies he controlled. Prigozhin is widely reported to have ties to Russian President Vladimir Putin, 
In mid-2014, the IRA sent employees to the United States on an intelligence-gathering mission with instructions  || lown?"C01"8081066
1052 F.S.
385014 McGamm 8/12/17 302, at 2-43 9:4w up on way Coril'ss, DCIn the Trump Transcorires .
1169 Kpolyking Adair:24903 Ucited Summet at S.DCDAIN0.: 1731 Whree've
549 Junt 20166, Whice Hector RosS09-SJESe-JAGIDZLC5.CL PJOP-SCAINWJI POPSEAN IRAu_USSaaskinrmE Soferiw?030 YatexAr, he deputors." 869 Evenyagozzey?/gr. Trump.
Gatobhit Mur. Regaged, Love PUknize-

and entities involved in the social media campaign have been charged with participating in a conspiracy to defraud the United States by undermining through deceptive acts the work of federal agencies charged with regulating foreign influence in U.S. elections, as well as related counts of identity theft. See United States v. Internet Research Agency, et al. , No. 18-cr-32 D.D.C.. Separately, Russian intelligence officers who carried out the hacking into Democratic Party computers and the personal email accounts of individuals affiliated with the Clinton Campaign conspired to violate, among other federal laws, the federal computer-intrusion statute, and they have been so charged. See United States v. Netyksho, et al. , No. 18-cr-215 D.D.C..  || Secceining Prince to contact IIs, information statfformed Darry in tourd.
U.S. agent-reseltchs with you Trump proypeds acted, Special

### : Temperature == .6 prediction
and entities involved in the social media campaign have been charged with pa

 investigation of Russian election interference. The Office charged some of those lies as violations of the federal false-statements statute. Former National Security Advisor Michael Flynn pleaded guilty to lying about his interactions with Russian Ambassador Kislyak during the transition period. George Papadopoulos, a foreign policy advisor during the campaign period, pleaded guilty to lying to investigators about, inter alia , the nature and timing of his interactions with Joseph Mifsud, the professor who told Papadopoulos that the Russians had dirt on candidate Clinton in the form of thousands of emails. Former Trump Organization attorney Michael Cohen pleaded guilty to making false statements to Congress about the Trump Moscow project.  || The President Accessal of Iffyers, information that the President Trump said Party about the Compution , 158 Manaforts Deralion Times New York Times January 2014, Russian Alcomare ban and Russian Emile

### : Temperature == 1.3 prediction
 invest

 and with other political activists to seek to coordinate political activities, including the staging of political rallies. 5 The investigation did not identify evidence that any U.S. persons knowingly or intentionally coordinated with the IRAs interference operation.
By the end of the 2016 U.S. election, the IRA had the ability to reach millions of U.S. persons through their social media accounts. Multiple IRA-controlled Facebook groups and
2 The Office is aware of reports that other Russian entities engaged in similar active measures operations targeting the United States. Some evidence collected by the Office corroborates those reports, and the Office has shared that evidence with other offices in the Department of Justice and FBI.
3  || She bus deection of
983 Gorcov 3/13/17 302, at 3.
909 Cohen 12/12/18 3

### : Temperature == .6 prediction
 and with other political activists to seek to coordinate political activities, including the staging of political rallies. 5 The investigatio

### 400 Sequence Length Model


#### Parameters

In [37]:
learning_rate_400 = 5e-4
embedding_dim_400 = 256
nb_epoches_400 = 1
rnn_units_400 = 600
load_weights_400 = True
keep_training_400 = False
examples_per_epoch_400 = len(sub_text)//seq_length_400
steps_per_epoch_400 = int(examples_per_epoch_400 / BATCH_SIZE)


dataset_sb_400 = dataset_400.shuffle(BUFFER_SIZE).batch(BATCH_SIZE, drop_remainder=True)
print(dataset_sb_400)

<DatasetV1Adapter shapes: ((64, 400), (64, 400)), types: (tf.int64, tf.int64)>


#### Set up Checkpoint Path

In [38]:
if drive_load == True:
    checkpoint_400_path = "/content/gdrive/My Drive/code/redaction model/checkpoints_400_model/epochs:{epoch:03d}-loss:{loss:.3f}.hdf5"
else:
    checkpoint_400_path = "checkpoints_400_model/epochs:{epoch:03d}-loss:{loss:.3f}.hdf5"
    
checkpoint_400_dir = os.path.dirname(checkpoint_400_path)

if not os.path.isdir(checkpoint_400_dir):
    os.mkdir(checkpoint_400_dir)
    
print(checkpoint_400_dir)

checkpoints_400_model


#### Build Model

In [39]:
def build_model_400(vocab_size, embedding_dim, rnn_units, batch_size):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim,
                              batch_input_shape=[batch_size, None]),
        tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.5),         
        tf.keras.layers.GRU(rnn_units,
                        return_sequences=True,
                        stateful=True,
                        recurrent_initializer='glorot_uniform'),
        tf.keras.layers.Dropout(0.5), 
        tf.keras.layers.Dense(2*vocab_size),
        tf.keras.layers.Dense(vocab_size)])
    return model

In [40]:
model_400 = build_model_400(
    vocab_size = len(vocab),
    embedding_dim=embedding_dim_400,
    rnn_units=rnn_units_400,
    batch_size=BATCH_SIZE)

adam_opt = Adam(lr=learning_rate_400)

model_400.compile(optimizer=adam_opt, loss=loss)

most_recent_400_checkpoint = max([checkpoint_400_dir + '/' + i for i in os.listdir(checkpoint_400_dir)], key = os.path.getctime)

if load_weights_400 == True:
    model_400.load_weights(most_recent_400_checkpoint)
    print("Weights loaded from : ", most_recent_400_checkpoint)

model_400.summary()


Weights loaded from :  checkpoints_400_model/epochs_029-loss_0.953.hdf5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_5 (Embedding)      (64, None, 256)           19456     
_________________________________________________________________
gru_5 (GRU)                  (64, None, 600)           1542600   
_________________________________________________________________
dropout_5 (Dropout)          (64, None, 600)           0         
_________________________________________________________________
gru_6 (GRU)                  (64, None, 600)           2161800   
_________________________________________________________________
dropout_6 (Dropout)          (64, None, 600)           0         
_________________________________________________________________
dense_10 (Dense)             (64, None, 152)           91352     
_________________________________________________________________
dens

#### Fit Model

In [41]:
checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(filepath=checkpoint_400_path, period=1,
                                                         monitor='loss', save_best_only=True, 
                                                         mode='min')

earlystop_callback = tf.keras.callbacks.EarlyStopping(monitor='loss', patience=3)

callbacks_list = [checkpoint_callback, earlystop_callback]

if load_weights_400 == True:
    if keep_training_400 == True:
        history = model_400.fit(
                            dataset_sb_400, 
                            epochs=nb_epoches_400, 
                            steps_per_epoch=steps_per_epoch_400, 
                            callbacks=callbacks_list)
    elif keep_training_400 == False:
        history = model_400.fit(
                            dataset_sb_400, 
                            epochs=1, 
                            steps_per_epoch=1, 
                            callbacks=callbacks_list)
else:
    history = model_400.fit(
                        dataset_sb_400, 
                        epochs=nb_epoches_400, 
                        steps_per_epoch=steps_per_epoch_400, 
                        callbacks=callbacks_list)

1/1 [==============================] - 24s 24s/step - loss: 0.9504


#### 1 Batch and Redactions

In [46]:
print(most_recent_400_checkpoint)

model_400_1batch = build_model_400(vocab_size, embedding_dim_400, rnn_units_400, batch_size=1)
model_400_1batch.load_weights(most_recent_400_checkpoint)
model_400_1batch.build(tf.TensorShape([1, None]))
model_400_1batch.summary()

print("400 Sequence Length predictors")
for index, redaction in enumerate(test_text_400[:10]):
    print("\n################################################\n")
    print(index)
    print("###")
    print(redaction)
    print("### : Temperature == 1 prediction")
    print(generate_text(model_400_1batch, start_string=redaction, temperature=1))
    print("### : Temperature == 0.6 prediction")
    print(generate_text(model_400_1batch, start_string=redaction, temperature=0.6))
    print("### : Temperature == 1.3 prediction")
    print(generate_text(model_400_1batch, start_string=redaction, temperature=1.3))
        

checkpoints_400_model/epochs_029-loss_0.953.hdf5
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_10 (Embedding)     (1, None, 256)            19456     
_________________________________________________________________
gru_14 (GRU)                 (1, None, 600)            1542600   
_________________________________________________________________
dropout_14 (Dropout)         (1, None, 600)            0         
_________________________________________________________________
gru_15 (GRU)                 (1, None, 600)            2161800   
_________________________________________________________________
dropout_15 (Dropout)         (1, None, 600)            0         
_________________________________________________________________
dense_20 (Dense)             (1, None, 152)            91352     
_________________________________________________________________
dense_21 (Dense)           

, the GRU began disseminating stolen materials through the fictitious online personas DCLeaks and Guccifer 2.0. The GRU later released additional materials through the organization WikiLeaks.
The presidential campaign of Donald J. Trump Trump Campaign or Campaign" showed interest in WikiLeakss releases of documents and welcomed their potential to damage candidate Clinton. Beginning in June 2016,  || Campaigns foreign palsp

################################################

3
###
■■■■■■■■■■■■■■ forecast to senior Campaign officials that WikiLeaks would release information damaging to candidate Clinton. WikiLeakss first release came in July 2016. Around the same time, candidate Trump announced that he hoped Russia would recover emails described as missing from a private server used by Clinton when she was Secretary of State he later said that he was speaking sarcastically. ■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■■

 the campaign period, pleaded guilty to lying to investigators about, inter alia , the nature and timing of his interactions with Joseph Mifsud, the professor who told Papadopoulos that the Russians had dirt on candidate Clinton in the form of thousands of emails. Former Trump Organization attorney Michael Cohen pleaded guilty to making false statements to Congress about the Trump Moscow project.  || 517 The next day, the President was interedeing him director.
95 Robert Goldstone :DEselaated enemyn. By Flynn pleaded guilty, puilieg it to candidate Trump than lubbind the Trump Campaign. 526 Dana B
### : Temperature == 0.6 prediction
 the campaign period, pleaded guilty to lying to investigators about, inter alia , the nature and timing of his interactions with Joseph Mifsud, the professor who told Papadopoulos that the Russians had dirt on candidate Clinton in the form of thousands of emails. Former Trump Organization attorney Michael Cohen pleaded guilty to making false statements to 